## 1. Load line charts dataset

In [15]:
from analyze_line_charts import *

df = load_line_rows() 

len(df)


640

## 2. Count how many unique questions exist in the line dataset

In [5]:
count_unique_questions(df)

The number of questions in the line dataset is 16


array(['What was the price of a barrel of oil in February 2015? Options: $57.36, $47.82, $50.24, $39.72, Omit',
       'In which month was the price of a barrel of oil the lowest in 2015? Options: March, May, July,December, Omit',
       'What was the price range of a barrel of oil in 2015? Options: $35 - $65, $48.36 - $60.95, $37.04 - $48.36, $37.04 - $60.95, Omit',
       'Over the course of the second half of 2015, the price of a barrel of oil was ____________. Options: rising, falling, staying, Omit',
       'About how much did the price of a barrel of oil fall from April to September in 2015? Options: $4, $15, $17, $45, Omit',
       'What was the average price of a pound of coffee beans in September 2013? Options: $4.9, $5.0, $5.1, $5.2, Omit',
       'When was the average price of a pound of coffee beans at minimum? Options: April 2013, September 2013, June 2014, December 2014, Omit',
       'What was the range of the average price of a pound of coffee beans between January 2013

## 3. Extract all rows for the selected target questions

In [11]:
target = "About how much did the price of a barrel of oil fall from April to September in 2015? Options: $4, $15, $17, $45, Omit"
matching_rows = get_matching_question_rows(df, target)

# 4. Summarize which colors are used in the 40 rows


In [9]:
color_usage_summary(matching_rows)

Colors used in the rows:
color_black              4
color_burlywood          4
color_chartreuse         4
color_lightgray          4
color_lightpink          4
color_mediumvioletred    4
color_navy               4
color_red                4
color_saddlebrown        4
color_tomato             4
dtype: int64


color_black              4
color_burlywood          4
color_chartreuse         4
color_lightgray          4
color_lightpink          4
color_mediumvioletred    4
color_navy               4
color_red                4
color_saddlebrown        4
color_tomato             4
dtype: int64

## 5. Extract all rows that use the color red


In [13]:
color_red_row = filter_color_red_rows(matching_rows)

color_red_row

     Unnamed: 0        dataset  \
160         480  line_data.csv   
161         481  line_data.csv   
180         540  line_data.csv   
181         541  line_data.csv   

                                              question answer  \
160  About how much did the price of a barrel of oi...    $15   
161  About how much did the price of a barrel of oi...    $15   
180  About how much did the price of a barrel of oi...    $15   
181  About how much did the price of a barrel of oi...    $15   

        question_type  referential_title  \
160  Make Comparisons                  1   
161  Make Comparisons                  1   
180  Make Comparisons                  0   
181  Make Comparisons                  0   

                                              filename  \
160  line_data.csv_line_Oil_Prices_Spike_Between_Ap...   
161  line_data.csv_line_Oil_Prices_Spike_Between_Ap...   
180  line_data.csv_line_Monthly_Oil_Price_History_i...   
181  line_data.csv_line_Monthly_Oil_Price_History_

,Unnamed: 0,dataset,question,answer,question_type,referential_title,filename,dataset_100_stacked_bar_data.csv,dataset_area_data.csv,dataset_bar_data.csv,dataset_histogram_data.csv,dataset_line_data.csv,dataset_pie_data.csv,dataset_scatterplot_data.csv,dataset_stacked_area_data.csv,dataset_stacked_bar_data.csv,plot_type_bar,plot_type_hist,plot_type_line,plot_type_pie,plot_type_scatter,plot_type_stacked_bar,plot_type_stacked_bar_100,color_black,color_burlywood,color_chartreuse,color_lightgray,color_lightpink,color_mediumvioletred,color_navy,color_red,color_saddlebrown,color_tomato
160,480,line_data.csv,About how much did the price of a barrel of oi...,$15,Make Comparisons,1,line_data.csv_line_Oil_Prices_Spike_Between_Ap...,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False
161,481,line_data.csv,About how much did the price of a barrel of oi...,$15,Make Comparisons,1,line_data.csv_line_Oil_Prices_Spike_Between_Ap...,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False
180,540,line_data.csv,About how much did the price of a barrel of oi...,$15,Make Comparisons,0,line_data.csv_line_Monthly_Oil_Price_History_i...,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False
181,541,line_data.csv,About how much did the price of a barrel of oi...,$15,Make Comparisons,0,line_data.csv_line_Monthly_Oil_Price_History_i...,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False
